# Tester Notebook for the Spreading Model Functionality

In [65]:
# Imports

from random import random
import pandas as pd
import networkx as nx

In [108]:
def sir_classical_spreading(G, beta = 1, gamma = None, starting_nodes = None):
    '''
    Run a SIR spreading process through a given network using classical triggering determined by parameter beta.
    Nodes stop attempting to infect neighbors after gamma time steps of being infected.
    
    Args:
        G (nx.Graph): Graph object to run spreading process on
        beta (float): probability of infected node infection neighbors
        gamma (int): time steps taken before node in I transitions to R (stops attempting to infect further)
        starting_nodes (int or list of ints): starting node(s) to use for spreading process

    Returns:
        results (pd.DataFrame): Data Frama containing time steps and corresponding fraction of nodes infected.
    '''

    # If no specific starting nodes given, use all nodes in G
    if starting_nodes == None:
        starting_nodes = list(G.nodes)

    # If only one starting node and given as int or str, contain in list
    elif type(starting_nodes) == int or type(starting_nodes) == str:
        starting_nodes = [starting_nodes]

    # Save amount of nodes in variable
    V = len(G.nodes)

    # Keep dictionary of fraction of infected and revomed nodes for each time step
    time_step_infection_rates = {0: [1/V]} # Time step 0 will always only have one infected node (the starting node)

    # Run complete spreading process for each starting node
    for s in starting_nodes:
        
        # Keep sets of nodes in each state
        S = set(G.nodes) - {s} # Initialize S as all nodes except starting node s
        I = {s} # Initialize I as only containing starting node
        R = set() # Initialize R as empty

        # Keep dict of nodes infected at each time step determine transition from I to R
        infected_at = {0: {s}} # Only starting node s is infected at time step 0

        # Time step iterator
        t = 1

        # Continue spreading process as long as I is not empty
        # Each iteration of this loop is the actions taken in time step t
        while I:

            # Keep set of newly infected nodes
            new_infected = set()

            # Iterate over all nodes in S to find new infections
            for u in S:

                # Check if u gets infected using classical trigger logic
                n = len(set(G.neighbors(u)) & I) # Amount of infected neighbors of u
                if random() < 1 - (1 - beta) ** n: 
                    new_infected.add(u)

            # Add new infected nodes to I and remove from S
            I = I.union(new_infected)
            S -= new_infected

            # Record infection time of newly infected nodes
            infected_at[t] = new_infected

            # Transition infected nodes from I to R after gamma time steps
            if t - gamma >= 0:
                to_transition = infected_at[t - gamma]
                R = R.union(to_transition)
                I -= to_transition

            # Add fraction of infected or removed nodes to time step record
            time_step_infection_rates.setdefault(t, []).append((len(I) + len(R)) / V)

            # Debug printing
            print(f't: {t}, |S|: {len(S)}, |I|: {len(I)}, |R|: {len(R)}')

            # Increment t
            t += 1

    # Average out values in dict of infection rates for each time step t
    time_step_infection_rates = {t: sum(i_r) / len(i_r) for t, i_r in time_step_infection_rates.items()}

    # Create Data Frame of infection rates for each time step t
    results = pd.DataFrame(list(time_step_infection_rates.items()), columns = ['t', 'IR'])

    return results

In [62]:
G = nx.read_edgelist('M1/edges.csv', delimiter = ',')
print(G)

Graph with 1429 nodes and 19357 edges


In [120]:
results = sir_classical_spreading(G, beta = .4, gamma = 3, starting_nodes = '0')

results

t: 1, |S|: 1401, |I|: 28, |R|: 0
t: 2, |S|: 943, |I|: 486, |R|: 0
t: 3, |S|: 339, |I|: 1089, |R|: 1
t: 4, |S|: 114, |I|: 1287, |R|: 28
t: 5, |S|: 32, |I|: 911, |R|: 486
t: 6, |S|: 4, |I|: 335, |R|: 1090
t: 7, |S|: 3, |I|: 111, |R|: 1315
t: 8, |S|: 3, |I|: 29, |R|: 1397
t: 9, |S|: 3, |I|: 1, |R|: 1425
t: 10, |S|: 3, |I|: 0, |R|: 1426


,t,IR
0,0,0.000700
1,1,0.019594
2,2,0.340098
3,3,0.762771
4,4,0.920224
5,5,0.977607
6,6,0.997201
7,7,0.997901
8,8,0.997901
9,9,0.997901
